***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [8]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-41ae  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [9]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [10]:
!touch log.txt
import logging
logging.basicConfig(level=logging.DEBUG,
                    filename='log.txt',
                    filemode='w',
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    force=True)
logger = logging.getLogger(__name__)

In [11]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
from timeit import timeit
import numpy as np
import json
import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 11 15:52 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [10]:
# from Invertedindex import *
# from factory import *


In [13]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [14]:
spark

In [15]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = '209100783-ir-project' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [16]:
parquetFile = spark.read.parquet(*paths)


In [17]:
doc_text_pairs = parquetFile.select("title", "id").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [18]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [19]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls 

Config.json	  factory.py		 reut_anchor_text.ipynb
Invertedindex.py  preprocess-data.ipynb


In [21]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/Config.json")
sc.addFile("/home/dataproc/Invertedindex.py")
sc.addFile("/home/dataproc/factory.py")

sys.path.insert(0,SparkFiles.getRootDirectory())

23/01/11 17:17:17 WARN org.apache.spark.SparkContext: The path /home/dataproc/Config.json has been added already. Overwriting of added paths is not supported in the current version.
23/01/11 17:17:17 WARN org.apache.spark.SparkContext: The path /home/dataproc/Invertedindex.py has been added already. Overwriting of added paths is not supported in the current version.
23/01/11 17:17:17 WARN org.apache.spark.SparkContext: The path /home/dataproc/factory.py has been added already. Overwriting of added paths is not supported in the current version.


In [22]:
from factory import *
from Invertedindex import *

In [23]:
import json
with open("Config.json","r") as file_config:
    config_dict = json.load(file_config)
    index_config = config_dict['index_configuration']
    proj_config = config_dict['project_configuration']
    print(config_dict)

{'comments': {'type_stemmer_comment': 'the type of stemmer maybe [porter,snowball,...]', 'type_index_comment': 'the type of index maybe [text,anchor_text,title]', 'type_tokenizer_comment': 'the type of tokenizer maybe [python,spacy,regex(self),regex(web)]', 'type_similarity_function_comment': 'the type of similarity_function maybe [fastcosine,cosine (reut),spacy,numpy,....]', 'TF_MASK': '2 ** 16 - 1 # Masking the 16 low bits of an integer'}, 'index_configuration': {'BLOCK_SIZE': 1999998, 'TUPLE_SIZE': 6, 'TF_MASK': 65535, 'NUM_BUCKETS': 124, 'tokenizer': 'regex', 'stemmer': ['porter', 'snowball', 'regular'], 'type_of_index': ['text', 'title', 'anchor']}, 'project_configuration': {'project_id': 'ir-2022-my', 'bucket_name': '209100783-ir-project', 'bucket_to_upload': 'testing-bucketing', 'file_to_download': 'multistream10_preprocessed.parquet'}}


In [24]:
Tok = FactoryIndex.get_tokenizer(index_config['tokenizer'])
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)

NUM_BUCKETS = index_config['NUM_BUCKETS']

def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS
stemmering = None
global_doc_size = {}

In [25]:
print(Tok)

In [41]:
# PLACE YOUR CODE HERE
def word_count(text, id,stemmer=None,idx_type=None):
    if stemmer is not None:
        logger.critical("this is stemmer")
        tokens = [stemmer.stem(token.group()) for token in Tok.tokenize(text.lower()) if token.group() not in all_stopwords]
    else:
        logger.critical("this is no stemmer")
        tokens = [token.group() for token in Tok.tokenize(text.lower()) if token.group() not in all_stopwords]

    token_countr = Counter(tokens)
  #calculating the vec size of the document
    if idx_type == 'text':
        global_doc_size[id] = np.linalg.norm(np.array(list(token_countr.values())))

  
    return [(word,(id,tf)) for word,tf in token_countr.items()]

def doc_to_count(text, id,stemmer=None,idx_type=None):
    if stemmer is not None:
        logger.critical("this is stemmer")
        tokens = [stemmer.stem(token.group()) for token in Tok.tokenize(text.lower()) if token.group() not in all_stopwords]
    else:
        logger.critical("this is no stemmer")
        tokens = [token.group() for token in Tok.tokenize(text.lower()) if token.group() not in all_stopwords]

    token_countr = Counter(tokens)
  
    doc_size = np.linalg.norm(np.array(list(token_countr.values())))
    doc_len = len(token_countr.values())

    
    return id,(np.round(doc_size,4),doc_len)


In [31]:
def reduce_word_counts(unsorted_pl,thr_len_pl, idx_type):
 
    if idx_type == 'text':
        if len(unsorted_pl) < thr_len_pl:
            return []
      
    return sorted(unsorted_pl,key=lambda tup: tup[1],reverse=True)


In [34]:
def partition_postings_and_write(postings,bucket_name,stem,index_type):

    rdd2 = postings.groupBy(lambda word: token2bucket_id(word[0])).map(lambda x : InvertedIndex.write_a_posting_list((x[0],x[1]),bucket_name,stem,index_type))
    return rdd2

In [42]:
def preprocess_data(index_type,stem,bucket_name,path,index_name_file,w2df_file_name,doc_statistics,stemmering):
    client = storage.Client()
    t_start = time()
    if index_type == 'text':
        doc_to_len = doc_text_pairs.map(lambda x:doc_to_count(x[0],x[1],stemmering,index_type)).collectAsMap()
        avg_tot_len = __builtins__.sum(map(lambda x: x[1],doc_to_len.values()))/len(doc_to_len.keys())
        with open(doc_statistics,'wb') as f:
            pickle.dump(doc_to_len,f)
            bucket = client.bucket(bucket_name)
            blob_posting_locs = bucket.blob(f"postings_gcp/{path}{doc_statistics}")
            blob_posting_locs.upload_from_filename(f"{doc_statistics}")
            del doc_to_len

    word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0],x[1],stemmering,index_type))
    postings = word_counts.groupByKey().mapValues(lambda x :reduce_word_counts(x,0, index_type))


    postings_filtered = postings.filter(lambda x: len(x[1])>0)

    w2df = postings_filtered.mapValues(lambda key: len(key))
    w2df_dict = w2df.collectAsMap()
  
    with open(w2df_file_name,'wb') as f:
        pickle.dump(w2df_dict,f)
        bucket = client.bucket(bucket_name)
        blob_posting_locs = bucket.blob(f"postings_gcp/{path}{w2df_file_name}")
        blob_posting_locs.upload_from_filename(f"{w2df_file_name}")
        del w2df_dict
    _ = partition_postings_and_write(postings_filtered,'testing-bucketing',stem,index_type).collect()
    print(time()-t_start)
  #-------------------------------------
  


In [62]:
def preprocess_data_for_anchor(index_type,stem,bucket_name,path,index_name_file,w2df_file_name,doc_statistics,stemmering,Tok):
    client = storage.Client()
    
    t_start = time()
    
    doc_text_pairs = doc_text_pairs_first.map(lambda x: (x[0],[(y[0],y[1]) for y in x[1]]))
    test_anchor = doc_text_pairs.flatMap(lambda x: word_count_anchor(x[1], Tok)).groupByKey()
    w_id_anchor = test_anchor.mapValues(lambda x: list(set(x)))
    
    w2df = w_id_anchor.mapValues(lambda key: len(key))
    w2df_dict = w2df.collectAsMap()
  
    with open(w2df_file_name,'wb') as f:
        pickle.dump(w2df_dict,f)
        bucket = client.bucket(bucket_name)
        blob_posting_locs = bucket.blob(f"postings_gcp/{path}{w2df_file_name}")
        blob_posting_locs.upload_from_filename(f"{w2df_file_name}")
        del w2df_dict
    _ = partition_postings_and_write(w_id_anchor,'testing-bucketing',stem,index_type).collect()
    print(time()-t_start)
  #-------------------------------------
  


In [57]:
def create_inverted_index(path,index_name_file,w2df_file_name,doc_statistics):
    inverted = InvertedIndex()
  

#    with open(w2df_file_name,'rb') as f:
#         w2df_dict = pickle.load(f)
#         inverted.df = w2df_dict
#     with open(doc_statistics,'rb') as f:
#         doc_stat = pickle.load(f)
#         inverted.global_doc_size = doc_stat


  

    super_posting_locs = defaultdict(list)
    for blob in client.list_blobs('testing-bucketing', prefix=f'postings_gcp/{path}'):
        if not blob.name.endswith("pickle"):
            continue
        with blob.open("rb") as f:
            posting_locs = pickle.load(f)
      
        for k, v in posting_locs.items():
        # todo change to super_posting_locs[k] = v
            super_posting_locs[k].extend(v)

    inverted.posting_locs = super_posting_locs

  # write the global stats out
    inverted.write_index('.', index_name_file,'testing-bucketing',path)


In [37]:

# index_type = 'text'
# stem = "regular"
# print(stem)
# if stem == 'regular':
#     stemmering = None
# else:
#     stemmering = FactoryIndex.get_stemmer(stem)
# client = storage.Client()
# bucket_name = 'testing-bucketing'
# path = f"{stem}-index/{index_type}-index/"
# index_name_file = f'{stem}_{index_type}_index'
# w2df_file_name = f"{stem}_{index_type}_w2df_statistics.stat"
# doc_statistics = f"{stem}_{index_type}_doc_statistics.stat"
# ###############################################
# preprocess_data(index_type,stem,bucket_name,path,index_name_file,w2df_file_name,doc_statistics,stemmering)
# ###############################################
# create_inverted_index(path,index_name_file,w2df_file_name,doc_statistics)
# ###############################################




In [36]:
doc_text_pairs = parquetFile.select("title", "id").rdd
index_type = 'title'
stem = "regular"
print(stem)
if stem == 'regular':
    stemmering = None
else:
    stemmering = FactoryIndex.get_stemmer(stem)
client = storage.Client()
bucket_name = 'testing-bucketing'
path = f"{stem}-index/{index_type}-index/"
index_name_file = f'{stem}_{index_type}_index'
w2df_file_name = f"{stem}_{index_type}_w2df_statistics.stat"
doc_statistics = f"{stem}_{index_type}_doc_statistics.stat"
###############################################
preprocess_data(index_type,stem,bucket_name,path,index_name_file,w2df_file_name,doc_statistics,stemmering)
###############################################
create_inverted_index(path,index_name_file,w2df_file_name,doc_statistics)
###############################################




regular


106.7861258983612


In [38]:

# index_type = 'text'
# stem = "snowball"
# print(stem)
# if stem == 'regular':
#     stemmering = None
# else:
#     stemmering = FactoryIndex.get_stemmer(stem)
# client = storage.Client()
# bucket_name = 'testing-bucketing'
# path = f"{stem}-index/{index_type}-index/"
# index_name_file = f'{stem}_{index_type}_index'
# w2df_file_name = f"{stem}_{index_type}_w2df_statistics.stat"
# doc_statistics = f"{stem}_{index_type}_doc_statistics.stat"
# ###############################################
# preprocess_data(index_type,stem,bucket_name,path,index_name_file,w2df_file_name,doc_statistics,stemmering)
# ###############################################
# create_inverted_index(path,index_name_file,w2df_file_name,doc_statistics)
# ###############################################





In [43]:
# doc_text_pairs = parquetFile.select("title", "id").rdd
index_type = 'title'
stem = "snowball"
print(stem)
if stem == 'regular':
    stemmering = None
else:
    stemmering = FactoryIndex.get_stemmer(stem)
client = storage.Client()
bucket_name = 'testing-bucketing'
path = f"{stem}-index/{index_type}-index/"
index_name_file = f'{stem}_{index_type}_index'
w2df_file_name = f"{stem}_{index_type}_w2df_statistics.stat"
doc_statistics = f"{stem}_{index_type}_doc_statistics.stat"
###############################################
preprocess_data(index_type,stem,bucket_name,path,index_name_file,w2df_file_name,doc_statistics,stemmering)
###############################################
create_inverted_index(path,index_name_file,w2df_file_name,doc_statistics)
###############################################



snowball


152.69418001174927


In [ ]:
# doc_text_pairs = parquetFile.select("title", "id").rdd
# index_type = 'text'
# stem = "porter"
# print(stem)
# if stem == 'regular':
#     stemmering = None
# else:
#     stemmering = FactoryIndex.get_stemmer(stem)
# client = storage.Client()
# bucket_name = 'testing-bucketing'
# path = f"{stem}-index/{index_type}-index/"
# index_name_file = f'{stem}_{index_type}_index'
# w2df_file_name = f"{stem}_{index_type}_w2df_statistics.stat"
# doc_statistics = f"{stem}_{index_type}_doc_statistics.stat"
# ###############################################
# preprocess_data(index_type,stem,bucket_name,path,index_name_file,w2df_file_name,doc_statistics,stemmering)
# ###############################################
# create_inverted_index(path,index_name_file,w2df_file_name,doc_statistics)
# ###############################################




In [45]:
doc_text_pairs = parquetFile.select("title", "id").rdd
index_type = 'title'
stem = "porter"
print(stem)
if stem == 'regular':
    stemmering = None
else:
    stemmering = FactoryIndex.get_stemmer(stem)
client = storage.Client()
bucket_name = 'testing-bucketing'
path = f"{stem}-index/{index_type}-index/"
index_name_file = f'{stem}_{index_type}_index'
w2df_file_name = f"{stem}_{index_type}_w2df_statistics.stat"
doc_statistics = f"{stem}_{index_type}_doc_statistics.stat"
###############################################
preprocess_data(index_type,stem,bucket_name,path,index_name_file,w2df_file_name,doc_statistics,stemmering)
###############################################
create_inverted_index(path,index_name_file,w2df_file_name,doc_statistics)
###############################################




porter


172.17224216461182


In [47]:
inverted = InvertedIndex.read_index('.',"regular_title_index")


In [58]:
# for i in inverted.posting_locs:
#     print(i)

In [59]:
# print(inverted.posting_locs['air'])

In [ ]:
#################### anchor #############################

In [60]:


def word_count_anchor(content, Tok, idx_type=None):

    tokens = []
    for id, text in content:
        tokens.extend([(token.group(),(id,1)) for token in Tok(text.lower()) if token.group() not in all_stopwords])

    return tokens

In [63]:
doc_text_pairs_first = parquetFile.select("id","anchor_text").rdd
# doc_text_pairs = doc_text_pairs_first.map(lambda x: (x[0],[(y[0],y[1]) for y in x[1]]))
# test_anchor = doc_text_pairs.flatMap(lambda x: word_count(x[1], Tok)).groupByKey()
# w_id_anchor = test_anchor.mapValues(lambda x: list(set(x)))

Tok = re.compile(r"[\#\@\w](['\-]?\w){2,24}",re.UNICODE).finditer
index_type = 'anchor'
stem = "regular"
print(stem)
if stem == 'regular':
    stemmering = None
else:
    stemmering = FactoryIndex.get_stemmer(stem)
client = storage.Client()
bucket_name = 'testing-bucketing'
path = f"{stem}-index/{index_type}-index/"
index_name_file = f'{stem}_{index_type}_index'
w2df_file_name = f"{stem}_{index_type}_w2df_statistics.stat"
doc_statistics = f"{stem}_{index_type}_doc_statistics.stat"
###############################################
preprocess_data_for_anchor(index_type,stem,bucket_name,path,index_name_file,w2df_file_name,doc_statistics,stemmering,Tok)
###############################################
create_inverted_index(path,index_name_file,w2df_file_name,doc_statistics)
###############################################

regular


830.4215822219849


In [91]:
import numpy as np
import math
def create_idf_dict(name_file_df, name_file_doc_len):

    with open(f'{name_file_df}', 'rb') as w2df:
        df_dict = pickle.load(w2df)

    with open(f'{name_file_doc_len}', 'rb') as d2size:
        doc_to_len = pickle.load(d2size)

    idf_dict ={}
    N = len(doc_to_len)
    for word, df in df_dict.items():
        idf_dict[word] = math.log2(N / df)
    return idf_dict

def doc_to_count_fix_idf(text, ide, stemmer=None, idx_type=None, idf_dict=None,Tok=None):

    if stemmer is not None:
        tokens = [stemmer.stem(token.group()) for token in Tok(text.lower()) if
                  token.group() not in all_stopwords]
    else:
        tokens = [token.group() for token in Tok(text.lower()) if token.group() not in all_stopwords]

    token_countr = Counter(tokens)
    if len(token_countr) == 0:
        return ide,()
    doc_len = len(token_countr.values())
    tf_idf_by_len = [(tf / doc_len) * idf_dict[w] for w, tf in token_countr.items() if w in idf_dict]
    doc_size_by_len = np.linalg.norm(np.array(tf_idf_by_len))
    
    max_word = token_countr.most_common(1)[0][1]
#     max_word = token_countr[most_com_word]
    tf_idf_by_max_word = [(tf / max_word) * idf_dict[w] for w, tf in token_countr.items() if w in idf_dict]
    doc_size_by_max_word = np.linalg.norm(np.array(tf_idf_by_max_word))
    
    doc_size_original_tf = np.linalg.norm(np.array(list(token_countr.values())))


    return np.int32(ide), (np.round(doc_size_by_len, 5), doc_len, max_word, np.round(doc_size_by_max_word, 5), np.round(doc_size_original_tf, 5))

In [66]:
!mkdir regular snowball porter

In [70]:
# !gsutil cp gs://testing-bucketing/postings_gcp/regular-index/text-index/*.stat regular/

Copying gs://testing-bucketing/postings_gcp/regular-index/text-index/regular_text_w2df_statistics.stat...
Copying gs://testing-bucketing/postings_gcp/regular-index/text-index/text_doc2len.stat...
Copying gs://testing-bucketing/postings_gcp/regular-index/text-index/text_doc2norm.stat...
Copying gs://testing-bucketing/postings_gcp/regular-index/text-index/text_doc2title.stat...
- [4 files][393.1 MiB/393.1 MiB]                                                
Operation completed over 4 objects/393.1 MiB.                                    


In [ ]:
# path = 'testing-bucketing/postings_gcp/regular-index/text-index'
# for blob in client.list_blobs('testing-bucketing', prefix=f'postings_gcp/{path}'):
#     if not blob.name.endswith("stat"):
#         continue
#     with blob.open("rb") as f:
#         posting_locs = pickle.load(f)

#     for k, v in posting_locs.items():
#     # todo change to super_posting_locs[k] = v
#         super_posting_locs[k].extend(v)

In [ ]:
doc_text_pairs = parquetFile.select("text", "id").rdd
Tok = re.compile(r"[\#\@\w](['\-]?\w){2,24}",re.UNICODE).finditer
index_type = 'text'
stem = "snowball"
print(stem)
if stem == 'regular':
    stemmering = None
else:
    stemmering = FactoryIndex.get_stemmer(stem)
client = storage.Client()
bucket_name = 'testing-bucketing'
path = f"{stem}-index/{index_type}-index/"
index_name_file = f'{stem}_{index_type}_index'
id_fixed_name = f"{stem}_{index_type}_doc2normlen_len_mwrd_normmxwrd_normoriginal.stat"
###############################################
# preprocess_data(index_type,stem,bucket_name,path,index_name_file,w2df_file_name,doc_statistics,stemmering)
###############################################
# create_inverted_index(path,index_name_file,w2df_file_name,doc_statistics)
idf_dict = create_idf_dict('snowball/snowball_text_w2df_statistics.stat','snowball/snowball-doc2len.stat')
id_fixed_rdd = doc_text_pairs.map(lambda x:doc_to_count_fix_idf(x[0],x[1],stemmering,index_type,idf_dict,Tok))
id_fixed_filter = id_fixed_rdd.filter(lambda x: len(x[1]) > 0 )
id_fixed = id_fixed_filter.collectAsMap()
# .collectAsMap()
with open(id_fixed_name,'wb') as f:
    pickle.dump(id_fixed,f)
    bucket = client.bucket(bucket_name)
    blob_posting_locs = bucket.blob(f"postings_gcp/{path}{id_fixed_name}")
    blob_posting_locs.upload_from_filename(f"{id_fixed_name}")
    del id_fixed
    
###############################################


regular


In [ ]:
doc_text_pairs = parquetFile.select("text", "id").rdd
Tok = re.compile(r"[\#\@\w](['\-]?\w){2,24}",re.UNICODE).finditer
index_type = 'text'
stem = "snowball"
print(stem)
if stem == 'regular':
    stemmering = None
else:
    stemmering = FactoryIndex.get_stemmer(stem)
client = storage.Client()
bucket_name = 'testing-bucketing'
path = f"{stem}-index/{index_type}-index/"
index_name_file = f'{stem}_{index_type}_index'
id_fixed_name = f"{stem}_{index_type}_doc2normlen_len_mwrd_normmxwrd_normoriginal.stat"
###############################################
# preprocess_data(index_type,stem,bucket_name,path,index_name_file,w2df_file_name,doc_statistics,stemmering)
###############################################
# create_inverted_index(path,index_name_file,w2df_file_name,doc_statistics)
idf_dict = create_idf_dict('snowball/snowball_text_w2df_statistics.stat','snowball/snowball-doc2len.stat')
id_fixed_rdd = doc_text_pairs.map(lambda x:doc_to_count_fix_idf(x[0],x[1],stemmering,index_type,idf_dict,Tok))
id_fixed_filter = id_fixed_rdd.filter(lambda x: len(x[1]) > 0 )
id_fixed = id_fixed_filter.collectAsMap()
# .collectAsMap()
with open(id_fixed_name,'wb') as f:
    pickle.dump(id_fixed,f)
    bucket = client.bucket(bucket_name)
    blob_posting_locs = bucket.blob(f"postings_gcp/{path}{id_fixed_name}")
    blob_posting_locs.upload_from_filename(f"{id_fixed_name}")
    del id_fixed
    
###############################################


snowball


23/01/11 21:21:17 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for broadcast_53_python !
23/01/11 21:21:17 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for broadcast_56_python !
23/01/11 21:21:17 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for broadcast_59_python !


In [ ]:
doc_text_pairs = parquetFile.select("text", "id").rdd
Tok = re.compile(r"[\#\@\w](['\-]?\w){2,24}",re.UNICODE).finditer
index_type = 'text'
stem = "snowball"
print(stem)
if stem == 'regular':
    stemmering = None
else:
    stemmering = FactoryIndex.get_stemmer(stem)
client = storage.Client()
bucket_name = 'testing-bucketing'
path = f"{stem}-index/{index_type}-index/"
index_name_file = f'{stem}_{index_type}_index'
id_fixed_name = f"{stem}_{index_type}_doc2normlen_len_mwrd_normmxwrd_normoriginal.stat"
###############################################
# preprocess_data(index_type,stem,bucket_name,path,index_name_file,w2df_file_name,doc_statistics,stemmering)
###############################################
# create_inverted_index(path,index_name_file,w2df_file_name,doc_statistics)
idf_dict = create_idf_dict('regular/regular_text_w2df_statistics.stat','regular/text_doc2len.stat')
id_fixed = doc_text_pairs.map(lambda x:doc_to_count_fix_idf(x[0],x[1],stemmering,index_type,idf_dict,Tok)).collectAsMap()
with open(id_fixed_name) as f:
    pickle.dump(id_fixed,f)
    bucket = client.bucket(bucket_name)
    blob_posting_locs = bucket.blob(f"postings_gcp/{path}{id_fixed_name}")
    blob_posting_locs.upload_from_filename(f"{id_fixed_name}")
    del id_fixed
    
###############################################


In [ ]:
doc_text_pairs = parquetFile.select("text", "id").rdd
Tok = re.compile(r"[\#\@\w](['\-]?\w){2,24}",re.UNICODE).finditer
index_type = 'text'
stem = "porter"
print(stem)
if stem == 'regular':
    stemmering = None
else:
    stemmering = FactoryIndex.get_stemmer(stem)
client = storage.Client()
bucket_name = 'testing-bucketing'
path = f"{stem}-index/{index_type}-index/"
index_name_file = f'{stem}_{index_type}_index'
id_fixed_name = f"{stem}_{index_type}_doc2normlen_len_mwrd_normmxwrd_normoriginal.stat"
###############################################
# preprocess_data(index_type,stem,bucket_name,path,index_name_file,w2df_file_name,doc_statistics,stemmering)
###############################################
# create_inverted_index(path,index_name_file,w2df_file_name,doc_statistics)
idf_dict = create_idf_dict('regular/regular_text_w2df_statistics.stat','regular/text_doc2len.stat')
id_fixed = doc_text_pairs.map(lambda x:doc_to_count_fix_idf(x[0],x[1],stemmering,index_type,idf_dict,Tok)).collectAsMap()
with open(id_fixed_name) as f:
    pickle.dump(id_fixed,f)
    bucket = client.bucket(bucket_name)
    blob_posting_locs = bucket.blob(f"postings_gcp/{path}{id_fixed_name}")
    blob_posting_locs.upload_from_filename(f"{id_fixed_name}")
    del id_fixed
    
###############################################


In [34]:
new_index = InvertedIndex().read_index('.',index_name_file)


this is creating the inverted index


In [44]:
# for idx,i in enumerate(new_index.global_doc_size.items()):
#     print(i)
#     if idx >= 1000:
#         break


(4045403, (119.0, 613))
(4045413, (27.5681, 111))
(4045419, (34.322, 362))
(4045426, (21.166, 100))
(4045432, (42.9767, 432))
(4045456, (77.9102, 444))
(4045466, (35.0, 340))
(4045471, (37.7094, 220))
(4045479, (103.9519, 784))
(4045516, (14.3178, 100))
(4045519, (14.5945, 65))
(4045523, (13.1909, 33))
(4045525, (15.2971, 141))
(4045532, (69.1375, 654))
(4045544, (44.3621, 278))
(4045546, (6.3246, 21))
(4045554, (49.1325, 338))
(4045575, (21.0, 111))
(4045577, (68.4105, 565))
(4045581, (24.8797, 82))
(4045584, (55.9017, 645))
(4045604, (15.843, 53))
(4045607, (35.9305, 267))
(4045640, (74.0203, 514))
(4045651, (14.0712, 35))
(4045659, (33.4365, 282))
(4045661, (35.4824, 359))
(4045672, (100.7422, 587))
(4045691, (33.4963, 160))
(4045698, (14.8324, 93))
(4045705, (19.3132, 59))
(4045710, (140.2213, 817))
(4045717, (108.6324, 720))
(4045722, (54.9363, 323))
(4045725, (22.0681, 147))
(4045760, (19.8242, 132))
(4045770, (28.688, 206))
(4045791, (18.2757, 117))
(4045792, (22.8692, 73))
(404

In [ ]:
# time the index creation time
t_start = time()
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered).collect()
index_const_time = time() - t_start

In [ ]:
# test index construction time
assert index_const_time < 60*120

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

Putting it all together

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index')
# upload to gs
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
!gsutil ls -lh $index_dst

# PageRank

**YOUR TASK (10 POINTS):** Compute PageRank for the entire English Wikipedia. Use your implementation for `generate_graph` function from Colab below.

In [ ]:
# Put your `generate_graph` function here

In [ ]:
t_start = time()
pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()

In [ ]:
# test that PageRank computaion took less than 1 hour
assert pr_time < 60*60

# Reporting

**YOUR TASK (5 points):** execute and complete the following lines to complete 
the reporting requirements for assignment #3. 

In [ ]:
# size of input data
!gsutil du -sh "gs://wikidata_preprocessed/"

14.28 GiB    gs://wikidata_preprocessed


In [ ]:
# size of index data
index_dst = f'gs://{bucket_name}/postings_gcp/'
!gsutil du -sh "$index_dst"

5.93 GiB     gs://sise_ir_assignment3/postings_gcp


In [ ]:
# How many USD credits did you use in GCP during the course of this assignment?
cost = 0 
print(f'I used {cost} USD credit during the course of this assignment')

**Bonus (10 points)** if you implement PageRank in pure PySpark, i.e. without using the GraphFrames package, AND manage to complete 10 iterations of your algorithm on the entire English Wikipedia in less than an hour. 


In [20]:
#If you have decided to do the bonus task - please copy the code here 

bonus_flag = False # Turn flag on (True) if you have implemented this part

t_start = time()

# PLACE YOUR CODE HERE

pr_time_Bonus = time() - t_start


In [21]:
# Note:test that PageRank computaion took less than 1 hour
assert pr_time_Bonus < 60*60 and bonus_flag